In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Define the paths to your dataset folders
train_dataset_dir = r"D:\datasets\Underwater_Image\LIMUC (Labeled Images for Ulcerative Colitis)\train_and_validation_sets\train_and_validation_sets"
test_dataset_dir = r"D:\datasets\Underwater_Image\LIMUC (Labeled Images for Ulcerative Colitis)\test_set\test_set"

# Initialize empty lists for X_train, Y_train, X_test, and Y_test
X_train = []
Y_train = []
X_test = []
Y_test = []

# Initialize an empty list to store categorical labels
categorical_labels = []

# Define a function to read and preprocess images
def process_images(folder_path, label, is_train_set=True):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith(".bmp"):  # Check if it's a file and ends with .bmp
            # Open and resize the image to (32, 32, 3)
            img = Image.open(file_path)
            img = img.resize((224, 224))
            img = img.convert("RGB")
            
            # Convert image data to a NumPy array
            img_array = np.array(img).astype('float32')  # Convert to float
            
            # Normalize the image data (optional)
            img_array /= 255.0  # Normalize pixel values to [0, 1]
            
            # Append the image data to the appropriate list
            if is_train_set:
                X_train.append(img_array)
                Y_train.append(label)  # Append the numerical label
            else:
                X_test.append(img_array)
                Y_test.append(label)  # Append the numerical label
            
            # Append the label for categorical encoding
            categorical_labels.append(label)  # Append the numerical label

# List the folders inside the training dataset directory
train_folders = os.listdir(train_dataset_dir)

# Create a label encoder for categorical labels
label_encoder = LabelEncoder()

# Loop through the training folders and process images
for label, folder_name in enumerate(train_folders):
    folder_path = os.path.join(train_dataset_dir, folder_name)
    if os.path.isdir(folder_path):  # Check if it's a directory
        process_images(folder_path, label)

# List the folders inside the test dataset directory
test_folders = os.listdir(test_dataset_dir)

# Loop through the test folders and process images
for label, folder_name in enumerate(test_folders):
    folder_path = os.path.join(test_dataset_dir, folder_name)
    if os.path.isdir(folder_path):  # Check if it's a directory
        process_images(folder_path, label, is_train_set=False)
        
        
        

# Encode Y_train and Y_test categorically
num_classes = len(np.unique(categorical_labels))
  # Update this to match the number of classes

# Convert Y_train and Y_test to NumPy arrays
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

# Make sure your labels are integers ranging from 0 to num_classes - 1
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# One-hot encode the labels
Y_train_categorical = to_categorical(Y_train, num_classes=num_classes)
Y_test_categorical = to_categorical(Y_test, num_classes=num_classes)



# # Encode Y_train and Y_test categorically using the label encoder
# num_classes = len(np.unique(Y_train))  # Automatically determine the number of classes
# Y_train_categorical = to_categorical(Y_train, num_classes=num_classes)
# Y_test_categorical = to_categorical(Y_test, num_classes=num_classes)


# Check the shape of X_train, Y_train_categorical, X_test, and Y_test_categorical
print("Shape of X_train:", np.shape(X_train))
print("Shape of Y_train_categorical:", np.shape(Y_train_categorical))
print("Shape of X_test:", np.shape(X_test))
print("Shape of Y_test_categorical:", np.shape(Y_test_categorical))

Shape of X_train: (9590, 224, 224, 3)
Shape of Y_train_categorical: (9590, 4)
Shape of X_test: (1686, 224, 224, 3)
Shape of Y_test_categorical: (1686, 4)


In [10]:
train_folders

['Mayo 0', 'Mayo 1', 'Mayo 2', 'Mayo 3']

In [4]:
from tensorflow import keras
model = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\mob_limuc_224_model.h5")

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)
# Compile the model
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
new_model.fit(np.array(X_train).astype('float32'), Y_train_categorical, batch_size=64, epochs=50, validation_split = .2)

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
120/120 [==============================] - 48s 267ms/step - loss: 0.5845 - accuracy: 0.7147 - val_loss: 9.3724 - val_accuracy: 0.0308
Epoch 2/50
120/120 [==============================] - 30s 253ms/step - loss: 0.5052 - accuracy: 0.7499 - val_loss: 9.2943 - val_accuracy: 0.0355
Epoch 3/50
120/120 [==============================] - 31s 258ms/step - loss: 0.4886 - accuracy: 0.7569 - val_loss: 9.0711 - val_accuracy: 0.0089
Epoch 4/50
120/120 [==============================] - 30s 254ms/step - loss: 0.4666 - accuracy: 0.7702 - val_loss: 8.6791 - val_accuracy: 0.0396
Epoch 5/50
120/120 [==============================] - 30s 248ms/step - loss: 0.4632 - accuracy: 0.7769 - val_loss: 10.7530 - val_accuracy: 0.0302
Epoch 6/50
120/120 [==============================] - 31s 256ms/step - loss: 0.4561 - accuracy: 0.7739 - val_loss: 9.2236 - val_accuracy: 0.0344
Epoch 7/50
120/120 [==============================] - 31s 262ms/step - loss: 0.4462 - accuracy: 0.7886 - val_loss: 10.2220 - val_

In [5]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
batch_size = 64
# Evaluate the model on the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test_categorical))
test_dataset = test_dataset.batch(batch_size)

test_loss, test_accuracy = new_model.evaluate(test_dataset)

# Make predictions on the test dataset
predictions = new_model.predict(test_dataset)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

# Calculate balanced accuracy
balanced_acc = balanced_accuracy_score(Y_test, predicted_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_test, predicted_labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Balanced Accuracy: {balanced_acc}")
print("Cohen Kappa Score:")
from sklearn.metrics import cohen_kappa_score
k = cohen_kappa_score(Y_test, predicted_labels, weights='quadratic')
print(k)

27/27 [==============================] - 5s 111ms/step - loss: 3.8580 - accuracy: 0.5724
Test Loss: 3.8579745292663574
Test Accuracy: 0.572360634803772
Accuracy: 0.5723606168446026
F1 Score: 0.5272983848005814
Balanced Accuracy: 0.34163618359739045
Cohen Kappa Score:
0.21757578366214403


In [2]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend
from keras import layers
from keras import utils
import keras
import tensorflow

class LayerScale(layers.Layer):
    """Layer scale module.

    References:
      - https://arxiv.org/abs/2103.17239

    Args:
      init_values (float): Initial value for layer scale. Should be within
        [0, 1].
      projection_dim (int): Projection dimensionality.

    Returns:
      Tensor multiplied to the scale.
    """

    def __init__(self, init_values, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.init_values = init_values
        self.projection_dim = projection_dim

    def build(self, input_shape):
        self.gamma = tensorflow.Variable(
            self.init_values * tensorflow.ones((self.projection_dim,))
        )

    def call(self, x):
        return x * self.gamma

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "init_values": self.init_values,
                "projection_dim": self.projection_dim,
            }
        )
        return config

model = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\conv_limuc_224_model.h5",custom_objects={ "LayerScale": LayerScale })

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(num_classes, activation='softmax')(x)
# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)
from keras.callbacks import EarlyStopping
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
new_model.fit(np.array(X_train).astype('float32'), Y_train_categorical, batch_size=8, epochs=50, validation_split = .2, callbacks=[early_stopping])

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
959/959 [==============================] - 1166s 1s/step - loss: 0.6776 - accuracy: 0.6590 - val_loss: 11.2546 - val_accuracy: 0.0000e+00
Epoch 2/50
959/959 [==============================] - 1169s 1s/step - loss: 0.6356 - accuracy: 0.6752 - val_loss: 11.9915 - val_accuracy: 0.0000e+00
Epoch 3/50
959/959 [==============================] - 1192s 1s/step - loss: 0.6337 - accuracy: 0.6752 - val_loss: 12.4448 - val_accuracy: 0.0000e+00
Epoch 4/50
959/959 [==============================] - 1195s 1s/step - loss: 0.6337 - accuracy: 0.6752 - val_loss: 12.1596 - val_accuracy: 0.0000e+00
Epoch 5/50
959/959 [==============================] - 1204s 1s/step - loss: 0.6330 - accuracy: 0.6752 - val_loss: 12.6865 - val_accuracy: 0.0000e+00
Epoch 6/50
959/959 [==============================] - 1202s 1s/step - loss: 0.6321 - accuracy: 0.6752 - val_loss: 13.4929 - val_accuracy: 0.0000e+00
Epoch 7/50
959/959 [==============================] - 1202s 1s/step - loss: 0.6328 - accuracy: 0.6752 - va

In [3]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
batch_size = 64
# Evaluate the model on the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test_categorical))
test_dataset = test_dataset.batch(batch_size)

test_loss, test_accuracy = new_model.evaluate(test_dataset)

# Make predictions on the test dataset
predictions = new_model.predict(test_dataset)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

# Calculate balanced accuracy
balanced_acc = balanced_accuracy_score(Y_test, predicted_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_test, predicted_labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Balanced Accuracy: {balanced_acc}")
print("Cohen Kappa Score:")
from sklearn.metrics import cohen_kappa_score
k = cohen_kappa_score(Y_test, predicted_labels, weights='quadratic')
print(k)

27/27 [==============================] - 23s 765ms/step - loss: 2.6233 - accuracy: 0.5486
Test Loss: 2.6233296394348145
Test Accuracy: 0.5486358404159546
Accuracy: 0.5486358244365361
F1 Score: 0.38873085990332895
Balanced Accuracy: 0.25
Cohen Kappa Score:
0.0
